In [1]:
import torch
import torchvision
import torch.nn as nn
import numpy as np
import torch.nn.functional as F

torch.manual_seed(199)



In [2]:
DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
DTYPE = torch.float16

In [3]:
random_affine = torchvision.transforms.RandomAffine(degrees=20,
                                                    scale=(0.8, 1.2),
                                                    translate=(0.2, 0.2),
                                                    interpolation=torchvision.transforms.InterpolationMode.BICUBIC)

release = torchvision.transforms.Resize((16,16))

In [4]:
transform1 = torchvision.transforms.Compose([random_affine,
                                             torchvision.transforms.Resize((32,32)),
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.ConvertImageDtype(torch.half)
                                             ])

transform2 = torchvision.transforms.Compose([torchvision.transforms.Resize((32,32)),
                                             torchvision.transforms.ToTensor(),
                                             torchvision.transforms.ConvertImageDtype(torch.half)
                                             ])

In [5]:
BATCH_SIZE = 128

In [6]:
trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform1)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=BATCH_SIZE,
                                          shuffle=True, num_workers=8, pin_memory=True)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform2)
testloader = torch.utils.data.DataLoader(testset, batch_size=BATCH_SIZE,
                                         shuffle=False, num_workers=8, pin_memory=True)

In [7]:
#Defining the convolutional neural network
class myLeNet5(nn.Module):
    def __init__(self):
        super(myLeNet5, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 4, kernel_size=5, stride=1, padding=0),
            #nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.layer2 = nn.Sequential(
            nn.Conv2d(4, 4, kernel_size=5, stride=1, padding=0),
            #nn.BatchNorm2d(4),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size = 2, stride = 2))
        self.fc = nn.Linear(4 * 5 * 5, 50)
        self.relu = nn.ReLU()
        #self.fc1 = nn.Linear(120, 84)
        #self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(50, 10)

    def forward(self, x):
        y = self.layer1(x)
        y = self.layer2(y)
        y = y.reshape(y.size(0), -1)
        y = self.fc(y)
        y = self.relu(y)
        #y = self.fc1(y)
        #y = self.relu1(y)
        y = self.fc2(y)
        return y

In [8]:
net = myLeNet5()
net = net.train().to(device=DEVICE,dtype=DTYPE)
optimizer = torch.optim.SGD(net.parameters(), lr=0.005, momentum=0.8)
loss_func = torch.nn.CrossEntropyLoss()

In [9]:
net = net.train().to(device=DEVICE)
for epoch in range(34):  # loop over the dataset multiple times

    running_loss = 0.0

    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs=inputs.to(device=DEVICE)
        labels=labels.to(device=DEVICE)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = loss_func(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        
    print('[%d] loss: %.3f' % (epoch + 1, running_loss))
    running_loss = 0.0

print('Finished Training')

RuntimeError: DataLoader worker (pid(s) 12880, 19332) exited unexpectedly

In [ ]:
mynet=net.eval()
all_counter=0
correct_counter=0
for i, data in enumerate(testloader, 0):
    inputs, labels = data
    inputs = inputs.to(DEVICE)
    labels = labels.to(DEVICE)
    out = mynet(inputs)
    out = out.detach().cpu().argmax(1)
    t = labels.cpu()
    for m in range(len(t)):
        all_counter += 1
        if t[m] == out[m]:
            correct_counter += 1

print(correct_counter, all_counter, correct_counter / all_counter)

In [ ]:
#2  max pool        0.9682
#2  avg pool+relu   0.9707
#2  max pool+relu   0.9772
#20 max pool+relu   0.9902
#20 去全连结 且120->80   0.9906
#25 去全连结 且120->80   0.9913
# 97.12

In [ ]:
print(mynet.cpu())

In [ ]:
print(mynet.state_dict())


In [ ]:
#torch.save(mynet,"mynet.pth")